## Overview

It's time to revisit the way table coordinates work.  In particular, the new design should support the following:

* The current API is inconsistent, and prevents numpy advanced indexing.

        table.column(0).width = 100
        table.column[0].width = 100
        table.column(0:3).width = 100
        table.column[0:3].width = 100
        table.width[:,0:3] = 100
        
        table.cell(0, 3).width = 100
        table.cell[0, 3].width = 100

* CellReferences:

        region.column[]
        region.row[]
        region.cell[]
        region.all?
        region.cells?

* Regions:
    * These are the well-defined regions:

            table.top.left
            table.top
            table.top.right
            table.left
            table.body
            table.right
            table.bottom.left
            table.bottom
            table.bottom.right
        
    * This is the special "catchall" region that accesses everything. We're holding on to it because it's useful for things like setting the alignment of every cell in a column, including the header.
        
            table.cells

    * These are aliases.  They should be deprecated to avoid confusion.

            table.header
            table.footer

* Columns and rows can be inserted into table coordinates.

* Table-aware plot embedding.
    * It should be possible to embed a bar plot or a line plot simply by assigning a special "formatter" to a row / column.
    * The layout of the embedded plot would be affected by cell spacing, gaps, padding, etc.
    * The tricky part will be handling formatters that get overridden / are discontiguous.
        
**TODO**

* When merging cells, merge their contents.
* Redesign indexing API.
* Add column / row insertion API.
* Restore the `axes(cell_padding)` attribute - or should there be separate top-right-bottom-left padding for every cell?
* Should row_offset and column_offset be handled with label styles?
* Implement data export.

In [1]:
import numpy
import toyplot.data

In [2]:
data = toyplot.data.Table()
data["a"] = numpy.array(["Foo-BAZ-46", "Bar-BAZ-46", "Foo-BAZ-23", "Bar-BAZ-23", "Foo-BLAH", "Bar-BLAH"])
data["b"] = numpy.array([0.381, 0.142, 0.473, 0.234, 0.165, 0.186])

In [3]:
canvas, table = toyplot.table(data, width=600, height=250)
table.grid.hlines[...] = "single"
table.grid.vlines[...] = "single"
#table.cells.column[0].lstyle = {"-toyplot-anchor-shift":"0.75in"}
table.cells.column[0].align = "center"

TypeError: 'instancemethod' object has no attribute '__getitem__'

a b Foo-BAZ-46 0 . 381 Bar-BAZ-46 0 . 142 Foo-BAZ-23 0 . 473 Bar-BAZ-23 0 . 234 Foo-BLAH 0 . 165 Bar-BLAH 0 . 186

In [ ]:
colormap = toyplot.color.CategoricalMap()

canvas = toyplot.Canvas(width=600, height=250)
table = canvas.table(rows=6, columns=4, label="Quarterly Report")
table.grid.hlines[...] = "single"
table.grid.vlines[...] = "single"
table.cells.column[0].width = 50
table.cells.column[1].data = data["a"]
table.cells.column[1].width = 90
table.cells.column[3].data = data["b"]
table.cells.column[3].width = 55
table.cells.column[3].format = toyplot.format.FloatFormatter(format="{:.2f}s")
table.cells.cell(0,0,rowspan=2).merge().data = "Red"
table.cells.cell(2,0,rowspan=2).merge().data = "Green"
table.cells.cell(4,0,rowspan=2).merge().data = "Blue"
axes = table.cells.cell(0, 2, rowspan=6).axes(show=True, yshow=False, padding=0, xlabel="Seconds")
axes.x.spine.show = False
axes.bars(data["b"][::-1], along="y", style={"stroke-width":8}, color=(numpy.array([0,1,0,1,0,1])[::-1], colormap));

In [ ]:
canvas = toyplot.Canvas(width=600, height=250)
table = canvas.table(rows=6, columns=4, label="Quarterly Report")
table.cells.column[0].width = 50
table.cells.column[1].data = data["a"]
table.cells.column[1].width = 90
table.cells.column[3].width = 55
table.cells.column[3].format = toyplot.format.FloatFormatter(format="{:.2f}s")
table.cells.cell(0,0,rowspan=2).merge().data = "Red"
table.cells.cell(2,0,rowspan=2).merge().data = "Green"
table.cells.cell(4,0,rowspan=2).merge().data = "Blue"
for index, value in enumerate(data["b"]):
    axes = table.cells.cell(index, 2).axes(padding=5, xmin=0, xmax=data["b"].max(), xlabel="Seconds", yshow=False)
    axes.bars([value], along="y", color=colormap.color(index % 2))
    axes.show = index == 5
    axes.x.spine.show = False